# Data Collection via Eikon API

In [1]:
# Imports & environment
# Core
import os, warnings
from datetime import datetime

# Data
import numpy as np
import pandas as pd

# Vendors / APIs
from dotenv import load_dotenv
import eikon as ek

# Excel export
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, Alignment, PatternFill, NamedStyle
from openpyxl.formatting.rule import ColorScaleRule
from openpyxl.chart import BarChart, Reference, ScatterChart, Series

warnings.filterwarnings("ignore", category=FutureWarning)

# Load env and set Eikon key
load_dotenv()
ek.set_app_key(os.getenv("EIKON_APP_KEY", "").strip())


## Set Tickers
Collect data via the tickers and save into a dataframe

In [26]:
# Universe via Screener (US primary) → tickers → Fundamentals (batched)

# Screener: all active, public, *primary* U.S. listings (covers XNAS/XNYS and other U.S. venues)
SCREENER_US = """
SCREEN(
  U(IN(Equity(active,public,primary))),
  IN(TR.ExchangeCountryCode,"US"),
  CURN=USD
)
"""

# Pull the screener universe (RICs + a few IDs for reference)
scr, err = ek.get_data(
    [SCREENER_US],
    ["TR.RIC","TR.CompanyName","TR.TickerSymbol","TR.ExchangeName","TR.ExchangeMarketIdCode"]
)

# Normalize and keep unique RICs
scr.rename(columns={
    "TR.RIC":"RIC",
    "TR.CompanyName":"Company Name",
    "TR.TickerSymbol":"Ticker",
    "TR.ExchangeName":"Exchange Name",
    "TR.ExchangeMarketIdCode":"MIC"
}, inplace=True)
scr = scr.dropna(subset=["RIC"]).drop_duplicates(subset=["RIC"]).reset_index(drop=True)

tickers = scr["RIC"].tolist()
print(f"Universe size (US primary): {len(tickers)}")

# Fundamentals fields
FIELDS = [
    "TR.TotalRevenue",
    "TR.EBITDA",
    "TR.NetIncome",
    "TR.NetIncomeInclExtraBeforeDistributions",
    "TR.TotalAssets",
    "TR.TotalEquity",
    "TR.CompanyMarketCap",
    "TR.TotalDebt",
    "TR.CashAndEquivalents"
]

# Batch fetch to avoid request size limits
def fetch_fundamentals_batch(rics, fields, batch=200):
    frames = []
    for i in range(0, len(rics), batch):
        chunk = rics[i:i+batch]
        df, e = ek.get_data(chunk, fields)
        if df is not None and not df.empty:
            frames.append(df)
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

data = fetch_fundamentals_batch(tickers, FIELDS, batch=200)
display(data.head())
print(f"Fundamentals rows: {len(data)}")


Universe size (US primary): 13636


C:\Users\chris\anaconda3\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
C:\Users\chris\anaconda3\Lib\site-packages\pandas\core\dtypes\cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


,Instrument,Total Revenue,EBITDA,Net Income Incl Extra Before Distributions,Total Assets,Total Equity,Company Market Cap,Total Debt,Cash and Equivalents
0,PLLVF.PK,0.0,-306940.0,2768040.0,37841510.0,37148160.0,27.395506,0.0,<NA>
1,DTRK.PK,7742570.0,392930.0,386800.0,8373470.0,1146320.0,20984174.64,2016410.0,3990550.0
2,MRCR.PK,52808520.0,2896630.0,406810.0,22792910.0,14862800.0,18969944.7,1919470.0,<NA>
3,HGYN.PK,245570.0,-94940.0,-98840.0,222660.0,-231450.0,2799028.725,0.0,46290.0
4,FN.N,3419327000.0,379283000.0,332527000.0,2831432000.0,1981812000.0,17444626821.099998,0.0,11183000.0


Fundamentals rows: 13636


## Clean Fields & Compute Valuation Metrics

Time to calculate some metrics from the data collected. Data wrangling step

In [27]:
# Rename Eikon display labels → concise names
rename_map = {
    "Total Revenue": "Revenue",
    "EBITDA": "EBITDA",
    "Net Income": "Net Income",
    "Net Income Incl Extra Before Distributions": "Net Income (Incl Extra)",
    "Total Assets": "Total Assets",
    "Total Equity": "Total Equity",
    "Company Market Cap": "Market Cap",
    "Total Debt": "Debt",
    "Cash and Equivalents": "Cash",
}
data = data.copy()
data.rename(columns={k: v for k, v in rename_map.items() if k in data.columns}, inplace=True)

# Coalesce Net Income (prefer standard; fallback to 'Incl Extra')
if "Net Income" not in data.columns and "Net Income (Incl Extra)" in data.columns:
    data["Net Income"] = data["Net Income (Incl Extra)"]
elif "Net Income" in data.columns and "Net Income (Incl Extra)" in data.columns:
    data["Net Income"] = pd.to_numeric(data["Net Income"], errors="coerce").fillna(
        pd.to_numeric(data["Net Income (Incl Extra)"], errors="coerce")
    )
if "Net Income (Incl Extra)" in data.columns:
    data.drop(columns=["Net Income (Incl Extra)"], inplace=True)

# Coerce numeric financials
for c in ["Revenue","EBITDA","Net Income","Total Assets","Total Equity","Market Cap","Debt","Cash"]:
    if c in data.columns:
        data[c] = pd.to_numeric(data[c], errors="coerce")

# Core metrics
data["Enterprise Value"] = data.get("Market Cap") + data.get("Debt") - data.get("Cash")

def safe_divide(n, d):
    n = pd.to_numeric(n, errors="coerce")
    d = pd.to_numeric(d, errors="coerce").replace({0: np.nan})
    return n / d

data["EV/EBITDA"] = safe_divide(data["Enterprise Value"], data["EBITDA"])
data["ROE"]       = safe_divide(data["Net Income"],       data["Total Equity"])
data["ROIC"]      = safe_divide(data["Net Income"],       data["Total Assets"])

# Friendly rounding for ratios
for c in ["EV/EBITDA","ROE","ROIC"]:
    if c in data.columns:
        data[c] = data[c].astype(float).round(2)

# Summary statistics kept in memory (will be moved to its own sheet)
summary = pd.DataFrame({
    "Metric": [c for c in ["EV/EBITDA","ROE","ROIC"] if c in data.columns],
    "Mean":   [data[c].mean()   for c in ["EV/EBITDA","ROE","ROIC"] if c in data.columns],
    "Median": [data[c].median() for c in ["EV/EBITDA","ROE","ROIC"] if c in data.columns],
    "Best":   [
        data["EV/EBITDA"].min() if "EV/EBITDA" in data else np.nan,
        data["ROE"].max()       if "ROE"       in data else np.nan,
        data["ROIC"].max()      if "ROIC"      in data else np.nan
    ],
    "Worst":  [
        data["EV/EBITDA"].max() if "EV/EBITDA" in data else np.nan,
        data["ROE"].min()       if "ROE"       in data else np.nan,
        data["ROIC"].min()      if "ROIC"      in data else np.nan
    ],
}).round(2)


# Save to Excel 
Using openpyxl, I saved the data into an xlsx file 

In [28]:
# Write main table to “Valuation Screener” 
filename = f"Valuation_Screener.xlsx"

header_row = 3
cols = [
    "Instrument","Revenue","EBITDA","Net Income","Total Assets","Total Equity",
    "Market Cap","Debt","Cash","Enterprise Value","EV/EBITDA","ROE","ROIC"
]
dollar_cols = {
    "Revenue","EBITDA","Net Income","Total Assets","Total Equity",
    "Market Cap","Debt","Cash","Enterprise Value"
}

wb = Workbook()
ws = wb.active
ws.title = "Valuation Screener"

# A1 headline KPI
total_cap_trn = round(pd.to_numeric(data.get("Market Cap"), errors="coerce").sum() / 1e12, 2)
ws["A1"] = f"Total Market Cap (USD Trn): {total_cap_trn}"
ws["A1"].font = Font(bold=True, size=12, color="1F4E78")

# Headers
header_fill = PatternFill(fill_type="solid", fgColor="1F4E78")
for i, col in enumerate(cols, start=1):
    cell = ws.cell(row=header_row, column=i, value=col)
    cell.font = Font(bold=True, color="FFFFFF")
    cell.fill = header_fill
    cell.alignment = Alignment(horizontal="center")
    ws.column_dimensions[cell.column_letter].width = 18
ws.freeze_panes = f"A{header_row+1}"

# Ensure expected columns exist
for c in cols:
    if c not in data.columns:
        data[c] = np.nan
present_cols = cols

# Styles
usd0 = NamedStyle(name="usd0"); usd0.number_format = '"$"#,##0'
rat2 = NamedStyle(name="rat2"); rat2.number_format = '0.00'
for s in (usd0, rat2):
    if s.name not in wb.named_styles:
        wb.add_named_style(s)

# Write table rows with alternate shading
df_xls = data[present_cols].astype(object).where(pd.notna(data[present_cols]), None).copy()
alt = PatternFill(fill_type="solid", fgColor="F2F2F2")

def to_python_scalar(v):
    return v.item() if isinstance(v, np.generic) else v

for r_idx, row in enumerate(df_xls.itertuples(index=False, name=None), start=header_row+1):
    for c_idx, value in enumerate(row, start=1):
        v = to_python_scalar(value)
        cell = ws.cell(row=r_idx, column=c_idx, value=v)
        col_name = present_cols[c_idx-1]
        if col_name in dollar_cols:
            cell.style = "usd0"
        elif col_name in {"EV/EBITDA","ROE","ROIC"}:
            cell.style = "rat2"
        if r_idx % 2 == 0:
            cell.fill = alt

wb.save(filename)


In [29]:
# Ranks & Screens (composite value + quality) → full universe on its own sheet
work = data.copy()

# Numeric coercion
for c in ["EV/EBITDA","ROE","ROIC","Market Cap","Enterprise Value","Debt","Cash","Revenue","EBITDA","Net Income"]:
    if c in work.columns:
        work[c] = pd.to_numeric(work[c], errors="coerce")

# Z-score helper (neutral if flat)
def zscore_series(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    mu = s.mean(skipna=True)
    sd = s.std(ddof=0, skipna=True)
    if pd.isna(sd) or sd == 0:
        return pd.Series(0.0, index=s.index)
    return (s - mu) / sd

# Composite score
work["z_ev_ebitda"] = -zscore_series(work.get("EV/EBITDA"))
work["z_roe"]       =  zscore_series(work.get("ROE"))
work["z_roic"]      =  zscore_series(work.get("ROIC"))
work["Score"] = 0.5*work["z_ev_ebitda"].astype(float) + 0.25*work["z_roe"].astype(float) + 0.25*work["z_roic"].astype(float)

# Rank and decile
score_for_rank = work["Score"].fillna(-np.inf)
work["Rank"] = score_for_rank.rank(ascending=False, method="min").astype(int)
pct = work["Rank"].rank(pct=True, method="max")
work["Decile"] = np.ceil(pct * 10).clip(1, 10).astype(int)

# Order and sort
screen_cols = [
    "Instrument","EV/EBITDA","ROE","ROIC","Score","Rank","Decile",
    "Market Cap","Enterprise Value","Debt","Cash","Revenue","EBITDA","Net Income"
]
screen_cols = [c for c in screen_cols if c in work.columns]
work = work[screen_cols].sort_values(["Rank","Score"], ascending=[True, False])

# Write to Excel
wb2 = load_workbook(filename)
if "Ranks & Screens" in wb2.sheetnames:
    del wb2["Ranks & Screens"]
ws_scr = wb2.create_sheet("Ranks & Screens")

header_fill = PatternFill(fill_type="solid", fgColor="1F4E78")
for j, col in enumerate(work.columns, start=1):
    cell = ws_scr.cell(row=1, column=j, value=col)
    cell.font = Font(bold=True, color="FFFFFF")
    cell.fill = header_fill
    ws_scr.column_dimensions[cell.column_letter].width = 18

usd0 = NamedStyle(name="usd0"); usd0.number_format = '"$"#,##0'
rat2 = NamedStyle(name="rat2"); rat2.number_format = '0.00'
for s in (usd0, rat2):
    if s.name not in wb2.named_styles:
        try:
            wb2.add_named_style(s)
        except ValueError:
            pass

def excel_value(v):
    if v is None:
        return None
    try:
        if pd.isna(v):
            return None
    except Exception:
        pass
    if isinstance(v, (np.floating, np.integer)):
        return float(v)
    return v

for r_idx, row in enumerate(work.itertuples(index=False), start=2):
    for c_idx, val in enumerate(row, start=1):
        colname = work.columns[c_idx-1]
        cell = ws_scr.cell(row=r_idx, column=c_idx, value=excel_value(val))
        if colname in {"Revenue","EBITDA","Net Income","Market Cap","Enterprise Value","Debt","Cash"}:
            cell.style = "usd0"
        elif colname in {"EV/EBITDA","ROE","ROIC","Score"}:
            cell.style = "rat2"

# Heatmap on Score
if "Score" in work.columns and len(work) > 0:
    score_col = work.columns.get_loc("Score") + 1
    ws_scr.conditional_formatting.add(
        f"{ws_scr.cell(row=2,column=score_col).coordinate}:{ws_scr.cell(row=1+len(work),column=score_col).coordinate}",
        ColorScaleRule(start_type="min", start_color="F8696B",
                       mid_type="percentile", mid_value=50, mid_color="FFEB84",
                       end_type="max", end_color="63BE7B")
    )

wb2.save(filename)


In [35]:
# Executive Summary sheet (KPIs + Top 10 and Bottom 10 by composite Score)
def to_numeric_series(s):
    return pd.to_numeric(s, errors="coerce")

def series_safe_mean(s):
    s = to_numeric_series(s)
    return float(s.mean()) if s.notna().any() else float("nan")

# Convert values to Excel-safe scalars
def excel_safe(v):
    # Fast path for None
    if v is None:
        return None
    # Handle pandas NA/NaN cleanly
    try:
        if pd.isna(v):
            return None
    except Exception:
        pass
    # Normalize NumPy scalars
    if isinstance(v, (np.floating, np.integer)):
        return float(v)
    if isinstance(v, np.bool_):
        return bool(v)
    # Timestamps → python datetime
    if isinstance(v, pd.Timestamp):
        return v.to_pydatetime()
    return v

exec_kpis = {
    "Universe (tickers)": int(len(data)),
    "Market cap covered (USD Trn)": round(float(to_numeric_series(data.get("Market Cap", pd.Series([]))).sum())/1e12, 2),
    "Median EV/EBITDA": float(to_numeric_series(data.get("EV/EBITDA", pd.Series([]))).median()),
}

k = min(10, len(work))
top10 = work.nsmallest(k, "Rank")
bot10 = work.nlargest(k, "Rank")

wb3 = load_workbook(filename)
if "Executive Summary" in wb3.sheetnames:
    del wb3["Executive Summary"]
ws_exec = wb3.create_sheet("Executive Summary")

# KPIs
ws_exec.cell(1, 1, "Executive Summary").font = Font(bold=True, size=12, color="1F4E78")
kpi_row = 3
for key, val in exec_kpis.items():
    ws_exec.cell(kpi_row, 1, key).font = Font(bold=True)
    cell = ws_exec.cell(kpi_row, 2, excel_safe(val))
    cell.number_format = '0.00' if isinstance(val, float) else '0'
    kpi_row += 1

# Top 10 block
start_top = kpi_row + 2
ws_exec.cell(start_top-1, 1, "Top 10 (Composite Score)").font = Font(bold=True, size=11, color="1F4E78")
for j, col in enumerate(top10.columns, start=1):
    ws_exec.cell(start_top, j, col).font = Font(bold=True)
for i, row in enumerate(top10.itertuples(index=False), start=start_top+1):
    for j, val in enumerate(row, start=1):
        ws_exec.cell(i, j, excel_safe(val))

# Bottom 10 block
start_bot = start_top + len(top10) + 3
ws_exec.cell(start_bot-1, 1, "Bottom 10 (Composite Score)").font = Font(bold=True, size=11, color="C00000")
for j, col in enumerate(bot10.columns, start=1):
    ws_exec.cell(start_bot, j, col).font = Font(bold=True)
for i, row in enumerate(bot10.itertuples(index=False), start=start_bot+1):
    for j, val in enumerate(row, start=1):
        ws_exec.cell(i, j, excel_safe(val))

wb3.save(filename)


In [36]:
# Summary Statistics sheet 

wb4 = load_workbook(filename)
if "Summary Statistics" in wb4.sheetnames:
    del wb4["Summary Statistics"]
ws_stat = wb4.create_sheet("Summary Statistics")

# Header row
for i, col in enumerate(summary.columns, start=1):
    ws_stat.cell(1, i, col).font = Font(bold=True)

# Body
for r, row in enumerate(summary.itertuples(index=False), start=2):
    for c, val in enumerate(row, start=1):
        out = val.item() if isinstance(val, np.generic) else val
        cell = ws_stat.cell(r, c, out)
        if isinstance(out, (int, float, np.number)):
            cell.number_format = '0.00'

wb4.save(filename)


In [53]:
# Create some charts

wb5 = load_workbook(filename)
if "Charts" in wb5.sheetnames:
    del wb5["Charts"]
ws_charts = wb5.create_sheet("Charts")

ws_src = wb5["Valuation Screener"]

# Column indices on source sheet
inst_idx = cols.index("Instrument") + 1 if "Instrument" in cols else None
ev_idx   = cols.index("EV/EBITDA") + 1   if "EV/EBITDA" in cols else None
roe_idx  = cols.index("ROE") + 1         if "ROE" in cols else None

data_start = header_row + 1
data_end   = ws_src.max_row

# Bar: EV/EBITDA by company
if inst_idx and ev_idx and data_end >= data_start:
    chart_bar = BarChart()
    chart_bar.title = "EV/EBITDA by Company"
    chart_bar.y_axis.title = "EV/EBITDA"
    cats = Reference(ws_src, min_col=inst_idx, min_row=data_start, max_row=data_end)
    vals = Reference(ws_src, min_col=ev_idx,   min_row=header_row, max_row=data_end)
    chart_bar.add_data(vals, titles_from_data=True)
    chart_bar.set_categories(cats)
    chart_bar.height, chart_bar.width = 15, 28
    ws_charts.add_chart(chart_bar, "A1")

# Scatter: ROE vs EV/EBITDA
if ev_idx and roe_idx and data_end >= data_start:
    chart_scatter = ScatterChart()
    chart_scatter.title = "Valuation Map (ROE vs EV/EBITDA)"
    chart_scatter.x_axis.title = "EV/EBITDA"
    chart_scatter.y_axis.title = "ROE"
    x_vals = Reference(ws_src, min_col=ev_idx,  min_row=data_start, max_row=data_end)
    y_vals = Reference(ws_src, min_col=roe_idx, min_row=data_start, max_row=data_end)
    series = Series(y_vals, x_vals)
    series.graphicalProperties.line.noFill = True
    series.marker.symbol = "circle"
    series.marker.size = 7
    chart_scatter.series.append(series)
    chart_scatter.legend = None
    chart_scatter.height, chart_scatter.width = 15, 28
    ws_charts.add_chart(chart_scatter, "A25")

wb5.save(filename)


In [54]:
# quick in-memory previews for debugging
preview_cols = [c for c in ["Instrument","EV/EBITDA","ROE","ROIC","Score","Rank","Decile","Market Cap"] if c in work.columns]
preview_top10 = work[preview_cols].head(10) if preview_cols else pd.DataFrame()
preview_bot10 = work[preview_cols].tail(10) if preview_cols else pd.DataFrame()


In [55]:
wb_final = load_workbook(filename)
for ws in wb_final.worksheets:
    ws.sheet_view.showGridLines = False
wb_final.save(filename)
